In [4]:
# scraper_newsapi.py (JSON Output Version)

import os
import requests
import json  # Import the json library instead of csv
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# Note: It's better to use the API key from your .env file
# API_KEY = os.getenv("NEWSAPI_KEY") 
# However, I will use the one you provided in the script for this example.
API_KEY = "68f0b4a61fe644c7b90bcc655c84f826"
BASE_URL = "https://newsapi.org/v2"

def fetch_news_by_query(query):
    """Fetches news based on a search query using the /everything endpoint."""
    endpoint = "/everything"
    
    headers = {
        'X-Api-Key': API_KEY # Use the API_KEY variable
    }
    
    params = {
        'q': query,
        'language': 'en',
        'sortBy': 'publishedAt',
        'pageSize': 20,
    }

    try:
        response = requests.get(BASE_URL + endpoint, headers=headers, params=params)
        response.raise_for_status()
        
        data = response.json()
        if data.get('status') != 'ok':
            print(f"API Error: {data.get('message')}")
            return []
            
        # The 'articles' key already contains a list of dictionaries, which is perfect for JSON
        return data.get('articles', [])
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for query '{query}': {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

# --- REPLACED FUNCTION ---
# The save_articles_to_csv function is replaced by save_articles_to_json
def save_articles_to_json(articles, query):
    """Saves a list of NewsAPI articles to a JSON file."""
    if not articles:
        print("No articles to save.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    # Use the .json file extension
    filename = f"newsapi_{query.replace(' ', '_')}_{timestamp}.json"

    try:
        with open(filename, 'w', encoding='utf-8') as jsonfile:
            # Use json.dump() to write the Python list to a JSON file
            # 'indent=4' makes the file human-readable
            # 'ensure_ascii=False' allows for special characters (like accents) to be saved correctly
            json.dump(articles, jsonfile, ensure_ascii=False, indent=4)
            
        print(f"Successfully saved {len(articles)} articles to {filename}")
    except IOError as e:
        print(f"Error writing to JSON file: {e}")

# Main execution block
if __name__ == "__main__":
    search_query = input("Enter a search query for news: ").strip()
    
    if not search_query:
        print("Search query cannot be empty.")
    else:
        print(f"Searching for news about: '{search_query}' using NewsAPI.org")
        
        # 1. Fetch the news
        found_articles = fetch_news_by_query(search_query)
        
        # 2. Save the news to a JSON file (updated call)
        if found_articles:
            save_articles_to_json(found_articles, search_query)
        else:
            print("Could not find any articles for the given query.")

Searching for news about: 'Google' using NewsAPI.org
Successfully saved 20 articles to newsapi_Google_20250621_141313.json
